In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [5]:
## Read The dataset
training = spark.read.csv('test_ml.csv',header=True,inferSchema=True)

In [6]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [9]:
#[Age,Experience]----> new feature--->independent feature

In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [11]:
output=featureassembler.transform(training)

In [12]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [13]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [14]:
finalized_data=output.select("Independent Features","Salary")

In [15]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [20]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary',regParam=0.1)
regressor=regressor.fit(train_data)

In [21]:
### Coefficients
regressor.coefficients

DenseVector([1768.316, -294.4097])

In [22]:
### Intercepts
regressor.intercept

-21838.181059782793

In [23]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [24]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [29.0,4.0]| 20000| 28265.34278853076|
|          [30.0,8.0]| 25000|28856.020030053434|
+--------------------+------+------------------+



In [25]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(6060.681409292098, 41592390.94204538)

In [27]:
pred_results.r2

-5.654782550727261